In [1]:
from proteinworkshop.features.factory import ProteinFeaturiser
from proteinworkshop.datasets.utils import create_example_batch
#from proteinworkshop.models.graph_encoders.schnet import SchNetModel
#from torch.utils.data import DataLoader
from torch_geometric.loader import DataLoader
import pickle
from transmembraneDataset import transmembraneDataset
import transmembraneUtils as tmu
import torch
from transmembraneModels import GraphEncDec 
from transmembraneModels import init_linear_weights
import os 
from torch_geometric.utils import unbatch
import wandb
import numpy as np 
import torchmetrics

cfg_path = None

if(cfg_path==None):
  BATCH_SZ = 1
  LR = 1e-04
  GCSIZE = 64
  LSTMSIZE = 64
  DROPOUT = 0.0
  N_EPOCHS = 10
  DECODER = "LSTMB"
  NUM_SAMPLES = 64
  EXPERIMENTTYPE = "OVERFIT"
  SPLITTYPE = "PROTOTYPE"
  FEATURISERFLAG = "SIMPLE"
  SAVERESULTS = True
  WEIGHTDECAY = 0.0
  param_cfg = {"experimentType": EXPERIMENTTYPE,"SplitType":SPLITTYPE,"NUM_SAMPLES":NUM_SAMPLES,"Featuriser":FEATURISERFLAG,"DecoderType":DECODER,"LR":LR,"WEIGHTDECAY":WEIGHTDECAY,"GCSize":GCSIZE,"LSTMSize":LSTMSIZE,"Dropout":0.0,"BATCH_SZ":BATCH_SZ,"N_EPOCHS":N_EPOCHS,"Save results":SAVERESULTS}
  TRACKING = False


/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


[11/28/23 14:48:44] DEBUG    No `.env` file found in project root. Checking for env vars...         ]8;id=188704;file:///home/max/.local/lib/python3.10/site-packages/proteinworkshop/constants.py\constants.py]8;;\:]8;id=932256;file:///home/max/.local/lib/python3.10/site-packages/proteinworkshop/constants.py#22\22]8;;\

                    DEBUG    No env var `DATA_PATH` found. Setting default...                       ]8;id=105396;file:///home/max/.local/lib/python3.10/site-packages/proteinworkshop/constants.py\constants.py]8;;\:]8;id=337640;file:///home/max/.local/lib/python3.10/site-packages/proteinworkshop/constants.py#28\28]8;;\

                    INFO     DATA_PATH:                                                             ]8;id=699700;file:///home/max/.local/lib/python3.10/site-packages/proteinworkshop/constants.py\constants.py]8;;\:]8;id=771536;file:///home/max/.local/lib/python3.10/site-packages/proteinworkshop/constants.py#39\39]8;;\
                             /home/max/.local/lib/python3.10/site-packages/proteinworkshop/data                    

Make example batch

In [2]:
import graphein.protein.tensor as gpt
from torch_geometric.data import Batch
import numpy as np 
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

#"3eiy", "4hhb", "1a0q"

data1 = gpt.io.protein_to_pyg(
    pdb_code="4hhb", # Can alternatively pass a path or a uniprot ID (for AF2) with path=... and uniprot_id=...
    chain_selection=["A", "B", "C", "D"], # Select all 4 chains
    deprotonate=True, # Deprotonate the structure
    keep_insertions=False, # Remove insertions
    keep_hets=[], # Remove HETATMs
    model_index=1, # Select the first model
    # Can select a subset of atoms with atom_types=...
    )

data2 = gpt.io.protein_to_pyg(
    pdb_code="3eiy", # Can alternatively pass a path or a uniprot ID (for AF2) with path=... and uniprot_id=...
    chain_selection=["A", "B", "C", "D"], # Select all 4 chains
    deprotonate=True, # Deprotonate the structure
    keep_insertions=False, # Remove insertions
    keep_hets=[], # Remove HETATMs
    model_index=1, # Select the first model
    # Can select a subset of atoms with atom_types=...
    )

def featurise_protein(prot):
    protein = gpt.Protein().from_data(prot) #load protein from pdb-data
    #add edges etc
    protein.batch = protein.coords
    protein.edges("knn_8",cache="edge_index")
    protein.pos = protein.coords[:,1,:]
    protein.x = protein.residue_type
    return protein.to_data() #returns a torch geometric data object used for batching https://github.com/a-r-j/graphein/blob/master/graphein/protein/tensor/data.py#L266



#print("Data: ", data1)
#print("ID: ", data1.id)
#print("Residues: ", data1.residues[:10])
#print("Residue IDs: ", data1.residue_id[:10])
data1 = featurise_protein(data1)
data2 = featurise_protein(data2)


#batch = Batch.from_data_list([data1, data2])
trainloader = DataLoader([data1,data2],batch_size = 2)


In [3]:
import torch


featuriser = ProteinFeaturiser( #note: input is a protein Batch
          representation="CA",
          scalar_node_features=["amino_acid_one_hot"],
          vector_node_features=[],
          edge_types=["knn_16"],
          scalar_edge_features=["edge_distance"],
          vector_edge_features=[],
        )

model = GraphEncDec(featuriser=featuriser, n_classes=6,hidden_dim_GCN=param_cfg["GCSize"],decoderType=param_cfg["DecoderType"],LSTM_hidden_dim=param_cfg["LSTMSize"],dropout=param_cfg["Dropout"],LSTMnormalization = False)
model.apply(init_linear_weights) #change to xavier normal init

optimizer = torch.optim.Adam(model.parameters(),lr=param_cfg["LR"],weight_decay=param_cfg["WEIGHTDECAY"])
criterion = torch.nn.CrossEntropyLoss()

for i, data in enumerate(trainloader):
    model_output, encoder_output = model.forward(data)
    #print(out)


/home/max/.local/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning:

Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.



initialized lazy layers
Initialized LSTM layers
RECEIVED  DataBatch(fill_value=[2], chains=[748], coords=[748, 37, 3], residues=[2], x=[748], residue_id=[2], id=[2], atom_list=[2], batch=[748], edge_index=[2, 5984], residue_type=[748], pos=[748, 3], ptr=[3])
Encoder outputs:  dict_keys(['node_embedding', 'graph_embedding'])
LSTM INPUT SHAPE:  torch.Size([748, 64])
LSTM OUTPUT SHAPE:  torch.Size([748, 128])
LSTM HIDDEN SHAPE: 
LSTM projection shape:  torch.Size([748, 6])
